# Portfolio Class

This Portfolio class provides the a basic interface to interact with the Technical Analysis data of a set of symbols. It contains a set of *Symbol* objects and provides a similar set functionalities to the Symbol class:
<ul>   
    <li> Loading and saving the raw candlestick data from different sources (.csv, SQL..).</li>
    <li> Handling the data updating, missing data, separating trading sessions...</li>
    <li> Technical Analysis indicators</li>
    <li> Common plots.</li>
</ul>    

In adition, it also provides a set of functionalities for portfolio analysis.

### Import libraries

In [1]:
import sys
import datetime as dt
import pandas as pd
from IPython.display import Image
%matplotlib qt 

sys.path.insert(0,"..") # Adds higher directory to python modules path.

from traphing.data_classes import Velas, Symbol, Portfolio
from traphing.utils import Timeframes, unwrap
from traphing.graph.Gl import gl

## Creating a Symbol instance

When we create a Symbol object, we need to specify the basic parameters of the symbol data:
 - <b>symbol_names_list</b>: A list with the names of the symbols in the portfolio.
 -  <b>timeframes_list</b>: A list with the different timeframes of the candlesticks we want to include.

No candlestick data is loaded when creating the object.

In [2]:
portfolio_id = "Manu's winter portfolio"
symbol_names_list = ["AUDCHF", "AUDCAD"]
timeframes_list = [Timeframes.M15, Timeframes.D1]

portfolio = Portfolio(portfolio_id, symbol_names_list, timeframes_list)
portfolio

The Symbol objects are initialized with no candlestick data. They are stored in an internal dictionary where the key is the *symbol_name* and the value is the Symbol object. This implementation should be transparent to the usage of the Symbol class. 

We can access the Velas objects inside the Porfolio instance by their *symbol_name*. We can get the current symbol_names_list in the Porfolio object by means of the attribute *symbol_names_list* which has been modify using a *getter* to compute the list dinamically when accessed.

We can use the *Porfolio* object as a dictionary to get the Symbol objects.

In [3]:
portfolio.symbol_names_list

['AUDCHF', 'AUDCAD']

In [4]:
portfolio['AUDCHF']

### Adding and removing Symbol objects

We can easily add, and delete velas inside the object using the following methods.

In [5]:
portfolio.add_symbol("AUC", [])
print (portfolio.symbol_names_list)

['AUDCHF', 'AUDCAD', 'AUC']


In [6]:
portfolio.del_symbol("AUC")
print (portfolio.symbol_names_list)

['AUDCHF', 'AUDCAD']


## Loading the data from .csv

The Porfolio class makes it easy to load the candlestick data for all the symbols at once. It has the same loading functions as the Symbol class, which internally call the same function for all of its Symbol objects.

In [7]:
storage_folder = "../tests/data/storage/"
portfolio.load_data_from_csv(storage_folder)

Size ../tests/data/storage/M15/AUDCHF_M15.csv:  100400  rows
Size ../tests/data/storage/D1/AUDCHF_D1.csv:  4894  rows
Size ../tests/data/storage/M15/AUDCAD_M15.csv:  99483  rows
Size ../tests/data/storage/D1/AUDCAD_D1.csv:  4963  rows


## Set interval

We can set the time interval directly to the *Portfolio* object using the same interface as the one for Velas. The interval applied to the *Symbol* is applied to all of its Velas objects.

In [8]:
start_time = dt.datetime(2019,7,20)
end_time = dt.datetime(2019,8,20)

portfolio.set_time_interval(start_time, end_time, trim = False)

In [9]:
portfolio["AUDCHF"][Timeframes.M15].df

,Open,High,Low,Close,Volume
Timestamp,,,,,
2019-07-22 00:00:00,0.69208,0.69241,0.69173,0.69213,58.0
2019-07-22 00:15:00,0.69208,0.69240,0.69124,0.69203,187.0
2019-07-22 00:30:00,0.69209,0.69253,0.69167,0.69171,411.0
2019-07-22 00:45:00,0.69199,0.69232,0.69143,0.69185,182.0
2019-07-22 01:00:00,0.69209,0.69222,0.69142,0.69153,267.0
...,...,...,...,...,...
2019-08-20 22:45:00,0.66276,0.66282,0.66254,0.66261,456.0
2019-08-20 23:00:00,0.66261,0.66270,0.66250,0.66256,214.0
2019-08-20 23:15:00,0.66255,0.66288,0.66254,0.66266,326.0


## Symbol information

We can load all of the symbol information for all symbols in the portfolio at once using the 
This information is currently obtained from the MT5, storing it in a csv file with name: 
- Symbol_info.csv


In [10]:
symbols_properties_df = Symbol.load_symbols_properties_csv(storage_folder)
symbols_properties_df

,point_size,min_tick_value,contract_size,currency
symbol_name,,,,
GBPUSD,0.00001,1.000000,100000.0,GBP
USDCHF,0.00001,1.015641,100000.0,USD
USDJPY,0.00100,0.922152,100000.0,USD
USDCNH,0.00001,0.141363,100000.0,USD
USDRUB,0.00001,0.015618,100000.0,USD
...,...,...,...,...
NZDCHF,0.00001,1.015641,100000.0,NZD
NZDJPY,0.00100,0.922152,100000.0,NZD
NZDSGD,0.00001,0.732896,100000.0,NZD


In [11]:
portfolio.load_symbols_properties_from_df(symbols_properties_df)

## Estimating market hours 

In [12]:
portfolio.estimate_symbols_market_hours()

## Unwrapping the object

The following shows the unwrap of a Velas object. In it, you can appreciate that:
 - The start and end time of the interval are not stored at symbol level, just at velas level. This allows different timesticks to have different intervals.
 - The properties object contains all of the static information regarding the symbol, except for the market hours.
 - The MarketHours object contains the information about open market hours

In [13]:
unwrap(portfolio, "portfolio")

<Portfolio>	portfolio has children:
    <dict>	_symbols_dict

  <dict>	_symbols_dict has children:
      <Symbol>	AUDCHF
      <Symbol>	AUDCAD

    <Symbol>	AUDCAD has children:
        <str>	symbol_name:	AUDCAD
        <dict>	_velas_dict
        <SymbolProperties>	properties
        <MarketHours>	market_hours

      <MarketHours>	market_hours has children:
          <time>	open_time:	00:00:00
          <time>	close_time:	00:00:00
          <list>	trading_days_list
          <dict>	special_days_dict

        <dict>	special_days_dict has children:

        <list>	trading_days_list has children:
            <int>	trading_days_list[0]:	0
            <int>	trading_days_list[1]:	1
            <int>	trading_days_list[2]:	2
            <int>	trading_days_list[3]:	3
            <int>	trading_days_list[4]:	4

      <SymbolProperties>	properties has children:
          <str>	symbol_name:	AUDCAD
          <NoneType>	type:	None
          <NoneType>	country:	None
          <str>	currency:	AUD
     

## Indicators

The *Portfolio* class only implements indicators that need several candlestick data at timeframes to be computed. In order to obtain indicators for a given timeframe, use the Velas objects directly. 

NO INDICATORS IMPLEMENTED SO FAR

The interface for accessing

## Plottings

Like with the indicators, we only implement plotting functions that require different timeframes.

NO IMPLEMENTATION SO FAR